In [ ]:
from glob import glob
import numpy as np
import os
import cv2

#读取图像
melonL = cv2.imread('melonL.jpg',0)
melonR = cv2.imread('melonR.jpg',0)

d = 3*16 #disparity
b = 1#blockSize

#立体匹配：半全局块匹配算法
matcher0 = cv2.StereoSGBM_create(0,d, b, 24*b, 96*b, 12, 10, 50, 32, 63, cv2.STEREO_SGBM_MODE_SGBM_3WAY)
matcher1 = cv2.ximgproc.createRightMatcher(matcher0)

#创建滤波器
wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher0)
wls_filter.setLambda(80000)
wls_filter.setSigmaColor(1.3)
#计算视差图
disp0 = np.int16(matcher0.compute(melonL, melonR))
disp1 = np.int16(matcher1.compute(melonR, melonL))

#滤波器平滑视差图
depth = wls_filter.filter(disp0, melonL, None, disp1).astype(np.float32)/16.

depth = np.uint8(cv2.normalize(depth, depth, 255, 0, cv2.NORM_MINMAX))
cv2.imshow('Disparity', depth)
cv2.imwrite('Disparity.jpg', depth)
cv2.waitKey(0)
